# Regression testing in refactoring

We'll use regression testing while refactoring a software. Here, we'll aim for a complete rewrite of a sofware that parses file names of the form `data_1.dat` to `data_19.dat` and rewrites them to `data_001.dat`, ..., `data_019.dat`.

## First, we'll need some example file names

In [1]:
with open("filenames_orig.txt", mode="w") as f:
    for n in range(1, 19+1):
        f.write(f"data_{n}.dat\n")

!cat filenames_orig.txt

data_1.dat
data_2.dat
data_3.dat
data_4.dat
data_5.dat
data_6.dat
data_7.dat
data_8.dat
data_9.dat
data_10.dat
data_11.dat
data_12.dat
data_13.dat
data_14.dat
data_15.dat
data_16.dat
data_17.dat
data_18.dat
data_19.dat


## Then, we use an implementation in bash

We split files into three parts, add leading zeros to the number part and then reassemble the file name again.

In [2]:
%%bash

# define function that adds leading zeros
# to a file of the form data_11.dat:
function add_leading_zeros {
    stem=$(echo $1 | cut -d_ -f1);
    num=$(echo $1 | cut -d_ -f2 | cut -d. -f1);
    ext=$(echo $1 | cut -d. -f2);
    echo ${stem}_$(printf "%03d" ${num}).${ext};
}
export -f add_leading_zeros

# apply function to the list of file names
cat filenames_orig.txt | xargs -n1 -I {} bash -c "add_leading_zeros {}" > filenames_zeros.txt

# check output
cat filenames_zeros.txt

data_001.dat
data_002.dat
data_003.dat
data_004.dat
data_005.dat
data_006.dat
data_007.dat
data_008.dat
data_009.dat
data_010.dat
data_011.dat
data_012.dat
data_013.dat
data_014.dat
data_015.dat
data_016.dat
data_017.dat
data_018.dat
data_019.dat


## Let's take a minute here

At this point, we have the known input and the known output of a software: Two files containing the original and the corrected file names.

Next is the implementation of a new version of the software (taking an unusually drastic step of a complete rewrite) and checking that the new version reproduces the same output for known input as the old version.

In [3]:
%%file add_leading_zeros.py
#!/usr/bin/env python3
"""Add leading zeros to file names."""

import argparse

parser = argparse.ArgumentParser()
parser.add_argument("filename")
args = parser.parse_args()

def add_leading_zeros(filename):
    """Add leading zeros to a numbered file name.
    
    This will turn, e.g., data_11.dat into data_011.dat.
    """
    number = int(filename.split("_")[1].split(".")[0])

    return f"data_{number:03d}.dat"


print(add_leading_zeros(args.filename))

Writing add_leading_zeros.py


Now we have a file that, after making it executable, we can use to change file names:

In [4]:
!chmod 755 add_leading_zeros.py
!./add_leading_zeros.py --help

usage: add_leading_zeros.py [-h] filename

positional arguments:
  filename

optional arguments:
  -h, --help  show this help message and exit


## Run the test with known input and output

In [5]:
# apply function to the list of fileqnames
!cat filenames_orig.txt | xargs -n1 -I {} bash -c "./add_leading_zeros.py {}" > filenames_zeros_test.txt
!diff filenames_zeros.txt filenames_zeros_test.txt && echo "Test passed" || echo "Test FAILED"

Test passed


## Discussion

- There's many non-ideal choices we've made here. Can you name a few?

- Can you think of any occasion where regression testing could have helped during software development?